#Extreme Learning Machine associated with Syntetic Data Generator Prediction

##Full Code

In [ ]:
# The variables below describes if the notebook is no test mode or not,
# when "test", it means the data can be fewer than the original data.
# Result is not important, but the speed to test if it the notebook work is.

ENV_TYPE = "test" # "test" or "production"

In [1]:
import time
import torch
import numpy as np
import torch.nn as nn
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity


class ELMAutoEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size=1, use_gpu=True):
        super(ELMAutoEncoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.batch_size = batch_size
        self.use_gpu = use_gpu
        if self.use_gpu and torch.cuda.is_available():
            self.device = torch.device("cuda")
        else:
            self.device = torch.device("cpu")
            print("Warning: CUDA not available, using CPU.")
        self.set_random_seed()    
        self.weight = nn.Parameter(torch.randn(input_size, hidden_size, dtype=torch.float, device=self.device))
        self.bias = nn.Parameter(torch.randn(hidden_size, device=self.device))
        self.beta = nn.Parameter(torch.randn(hidden_size, output_size, device=self.device))
        self.activation_hidden = nn.ReLU()
        self.activation_output = nn.Identity()
        
    def forward(self, x):
        h = self.activation_hidden(x @ self.weight + self.bias)       
        y_pred = torch.sigmoid(h @ self.beta)  # apply sigmoid to output
        return y_pred

    def set_random_seed(self):
        seed = int(time.time())
        torch.manual_seed(seed)
        np.random.seed(seed)
    
    def fit(self, x_train,targets_train, n_neurons):
        H = self.activation_hidden(torch.matmul(x_train.float(), self.weight.float()) + self.bias.float())
     #  H = self.activation_hidden(torch.matmul(x_train.float(), self.weight) + self.bias)
        print('output of hiddenlayer',H.shape)
        H_inv = torch.pinverse(H)

        beta = torch.matmul(H_inv, targets_train.float())

        self.beta = nn.Parameter(beta)
        self.output_weight = nn.Parameter(self.beta)
        print('output weight matrix',self.output_weight.shape)
        self.bias = nn.Parameter(torch.zeros((n_neurons,)))
        return self
    
    def encode(self, x):
       # h = self.activation_hidden(x.float() @ self.weight.t())
        print('input to encoder',x.shape)
        h=self.activation_hidden(torch.matmul(x.float(), self.weight.float()) + self.bias.float())
        return h
    
    def decode(self, h):
        x_pred = self.activation_output(h @ self.output_weight)
        return x_pred
    
    def clear_memory(self):
        if self.use_gpu and torch.cuda.is_available():
            torch.cuda.empty_cache()
        else:
            torch.cuda.empty_cache()

def reconstruction_error(model,data,targets_train):
    with torch.no_grad():
        encoded = model.encode(data)
        print('encoded.shape',encoded.shape)
        decoded = model.decode(encoded)
        print('decoded',decoded.shape)
        mse_loss = nn.MSELoss()(decoded, targets_train)
        cos_sim = cosine_similarity(targets_train.reshape(targets_train.shape[0], -1), decoded.reshape(decoded.shape[0], -1))
        cos_dis=1 - cos_sim.mean()
        return mse_loss.item(),cos_dis

def load_prompts(file_path):
    with open(file_path) as f:
        prompts = json.load(f)
    return prompts

def prepare_data(prompts):
    text_features = []
    for prompt in prompts:
        text_feature = np.array(prompt['word_embedding'])
        text_features.append(text_feature)
    return text_features

def prepare_targets(prompts):
    clip_embeddings = []
    for prompt in prompts:
        clip_embedding = np.array(prompt['clip_embedding'])
        clip_embeddings.append(clip_embedding)
    return clip_embeddings

def EML_trained_with_vectors(matrix, neurons_in_hidden_layer=500, vectors_mean=0.0, vectors_variance=0.3, noise_mean=0.0, noise_variance=0.3, number_of_vectors=100):

    vectors_dimension = len(matrix) #the vector size must have the same number of columns of the matrix. 
    #that also implies that both the input and the output have the same result. 

    elm = ELMAutoEncoder(vectors_dimension, neurons_in_hidden_layer, vectors_dimension) #create ELM object. 

    vectorsArray = []
    outputArray = []
    for x in range(number_of_vectors): #generate a number of vectors defined in the function.

        #both vectors and noise have a specific mean and variance. 
        vector = np.random.normal(loc = vectors_mean, scale=vectors_variance, size=(1, vectors_dimension)) #generate vector
        noise = np.random.normal(loc = noise_mean, scale=noise_variance, size=(1, vectors_dimension)) #add noise to it
        vector += noise #sum it to the noise
        outputVector = np.dot(vector, matrix) #product with the matrix

        #append to vectors and outputs list to be saved for further testing.
        vectorsArray.append(vector)
        outputArray.append(outputVector)
 
        elm.fit(torch.from_numpy(vector).float(), torch.from_numpy(outputVector).float(), neurons_in_hidden_layer) #use the input and output to train the machine.

    print("Saving used vectors to train the machine in trained_vectors_input.npy and trained_vectors_output.npy")
    np.save('trained_vectors_input.npy', np.array(vectorsArray))
    np.save('trained_vectors_output.npy', np.array(outputArray))

    return elm #return the trained machine.

matrix = torch.tensor([[2, 3], [3,5]], dtype=torch.float32) #this is the matrix we're going to use. here it was manually generated.
elm = EML_trained_with_vectors(matrix, number_of_vectors=300) #the elm will be trained with vectors using this matrix. 

number_of_tests = 20 if ENV_TYPE == 'production' else 1
MSEList = []
cosList = []
for x in range(number_of_tests): #make test 20 times.
    #do the same vector generation process as before.
    testVector = torch.from_numpy(np.random.normal(loc = 0, scale=0.03, size=(1, len(matrix)))).float()
    noise = torch.from_numpy(np.random.normal(loc = 0, scale=0.03, size=(1, len(matrix)))).float()
    testVector+= noise
    outputVector = torch.matmul(testVector, matrix)

    #test its reconstruction error
    test_reconstruction_error, cos_dis = reconstruction_error(elm, testVector, outputVector)
    #append to list
    MSEList.append(test_reconstruction_error)
    cosList.append(cos_dis)

import statistics
#calculate and print statistics for it
print("average mse error for ", number_of_tests, "inputs: ", statistics.mean(MSEList))
print("stdev mse error for ", number_of_tests, "inputs: ", statistics.stdev(MSEList))

print("average cosine distance for ", number_of_tests, "inputs: ", statistics.mean(cosList))
print("stdev cosine distance for ", number_of_tests, "inputs: ", statistics.stdev(cosList))

output of hiddenlayer torch.Size([1, 500])
output weight matrix torch.Size([500, 2])
output of hiddenlayer torch.Size([1, 500])
output weight matrix torch.Size([500, 2])
output of hiddenlayer torch.Size([1, 500])
output weight matrix torch.Size([500, 2])
output of hiddenlayer torch.Size([1, 500])
output weight matrix torch.Size([500, 2])
output of hiddenlayer torch.Size([1, 500])
output weight matrix torch.Size([500, 2])
output of hiddenlayer torch.Size([1, 500])
output weight matrix torch.Size([500, 2])
output of hiddenlayer torch.Size([1, 500])
output weight matrix torch.Size([500, 2])
output of hiddenlayer torch.Size([1, 500])
output weight matrix torch.Size([500, 2])
output of hiddenlayer torch.Size([1, 500])
output weight matrix torch.Size([500, 2])
output of hiddenlayer torch.Size([1, 500])
output weight matrix torch.Size([500, 2])
output of hiddenlayer torch.Size([1, 500])
output weight matrix torch.Size([500, 2])
output of hiddenlayer torch.Size([1, 500])
output weight matrix t

##Explanation
This notebook uses the Extreme Learning Machine code imported from https://colab.research.google.com/drive/1rJLTQd97Ky6RcPLkDQGxYPuY7Uwq93Lo?usp=sharing#scrollTo=maYFj9jmz-dm to predict results of a Syntetic Data Generator.

The Syntetic Data Generator will create a random vector and add noise to it. Then, will multiply it by a matrix, as described by the following code:

In [ ]:
    matrix = torch.tensor([[2, 3], [3,5]], dtype=torch.double) #this is the matrix we're going to use.
    testVector = torch.from_numpy(np.random.normal(loc = 0, scale=0.03, size=(1, len(matrix))))#this is the generated vector.
    noise = torch.from_numpy(np.random.normal(loc = 0, scale=0.03, size=(1, len(matrix))))#this is the generated noise.
    testVector+= noise#this is the sum of both.
    outputVector = torch.matmul(testVector, matrix) #this is the dot product of vector and matrix.

Notice that it is necessary for the number of inputs of the vector to be equal to the number of columns of the matrix, since a dot product with them both will occur after adding the noise. The output will have the same size of the input vector as well.

To train the ELM, though, only one vector isn't enough. The function "EML_trained_with_vectors" will create an ELM object trained with a number of vectors generated like this.

In [ ]:
"""
Parameters:
matrix: input matrix that will be used as the dot product with all the vectors.
neurons_in_hidden_layer: number of neurons in the object.
vectors_mean: gaussian mean of the generated vectors.
vectors_variance: variance of the generated vectors.
noise_mean: gaussian mean of the generated noise vector.
noise_variance: variance of the generated vectors.
number_of_vectors: number of vectors used to train the machine.
"""
def EML_trained_with_vectors(matrix, neurons_in_hidden_layer=500, vectors_mean=0.0, vectors_variance=0.3, noise_mean=0.0, noise_variance=0.3, number_of_vectors=100):

    vectors_dimension = len(matrix) #the vector size must have the same number of columns of the matrix. 
    #that also implies that both the input and the output have the same result. 

    elm = ELMAutoEncoder(vectors_dimension, neurons_in_hidden_layer, vectors_dimension) #create ELM object. 

    vectorsArray = []
    outputArray = []
    for x in range(number_of_vectors): #generate a number of vectors defined in the function.

        #both vectors and noise have a specific mean and variance. 
        vector = np.random.normal(loc = vectors_mean, scale=vectors_variance, size=(1, vectors_dimension))
        noise = np.random.normal(loc = noise_mean, scale=noise_variance, size=(1, vectors_dimension))
        vector += noise
        outputVector = np.dot(vector, matrix)

        vectorsArray.append(vector)
        outputArray.append(outputVector)
 
        elm.fit(torch.from_numpy(vector), torch.from_numpy(outputVector), neurons_in_hidden_layer) #use the input and output to train the machine.
    print("Saving used vectors to train the machine in trained_vectors_input.npy and trained_vectors_output.npy")
    np.save('trained_vectors_input.npy', np.array(vectorsArray))
    np.save('trained_vectors_output.npy', np.array(outputArray))

    return elm #return the trained machine        

The trained vectors database is saved in the file trained_vectors_input.npy and trained_vectors_output.npy for further experiment.

After returning the trained machine, to test it, we generate a vector with the same way and compare its MSE and cosine distance. For a more precise experiment, this process will be repeated a number_of_tests so we can get a mean value of the metrics and a stdev. Those results are printed in the end.

In [ ]:
number_of_tests = 0
if ENV_TYPE == 'test':
    number_of_tests = 1
elif ENV_TYPE == 'production':
    number_of_tests = 20

MSEList = []
cosList = []
for x in range(number_of_tests):
    testVector = torch.from_numpy(np.random.normal(loc = 0, scale=0.03, size=(1, len(matrix))))
    noise = torch.from_numpy(np.random.normal(loc = 0, scale=0.03, size=(1, len(matrix))))
    testVector+= noise
    outputVector = torch.matmul(testVector, matrix)
    test_reconstruction_error, cos_dis = reconstruction_error(elm, testVector, outputVector)
    MSEList.append(test_reconstruction_error)
    cosList.append(cos_dis)

import statistics

print("average mse error for ", number_of_tests, "inputs: ", statistics.mean(MSEList))
print("stdev mse error for ", number_of_tests, "inputs: ", statistics.stdev(MSEList))

print("average cosine distance for ", number_of_tests, "inputs: ", statistics.mean(cosList))
print("stdev cosine distance for ", number_of_tests, "inputs: ", statistics.stdev(cosList))